In [ ]:

from pathlib import Path
from mllm.markov_games.rollout_tree import *
from mllm.markov_games.gather_and_export_utils import (
     gather_simulation_stats, 
     get_rollout_trees, get_rollout_tree_paths, gather_all_rewards, gather_all_chat_turns_for_path, get_all_agents
)
from mllm.markov_games.trust_and_split.tas_simulation import SplitsLog
import numpy as np
from statistics import mean
from collections import defaultdict
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import scienceplots
plt.style.use('dedestyle.mplstyle')


In [ ]:
# Get custom path from user input
custom_path = input("Enter the path to your rollout data: ").strip()

rollout_trees = get_rollout_trees(custom_path)
print(f"Rollout trees from {len(rollout_trees)} iterations loaded from: {custom_path}")

In [ ]:
main_rollouts_paths = [[get_rollout_tree_paths(root)[0] for root in roots] for roots in rollout_trees]

## Evolution of Greed

In [ ]:
def greedy_level(agent_id: str, sl: SimulationStepLog): 
    return sl.info["coins_given_to_self"][agent_id] / sl.info["num_coins"]

def split_filter(sl: SimulationStepLog):
    info : dict = sl.info
    if "coins_given_to_self" in info and "num_coins" in info:
        return True
    return False
nb_iterations = len(rollout_trees)
agent_ids = get_all_agents(rollout_trees[0][0])
print(agent_ids)
agent_greedy_levels = {}
fig, ax = plt.subplots()
for agent_id in agent_ids:
    stat_func = lambda sl: greedy_level(agent_id, sl)
    greedy_levels = [[gather_simulation_stats(path=path, filter=split_filter, stat_func=stat_func) for path in paths] for paths in main_rollouts_paths]
    greedy_levels = [np.mean(np.array(gls)) for gls in greedy_levels]
    ax.plot(np.arange(len(greedy_levels)), greedy_levels, label=agent_id)

# --- Axes & formatting ---
ax.set_xlabel("Iteration")
ax.set_ylabel("Average Coins Given to Self")

# Percent formatter — use xmax=100 if your stats are already in [0,100]
ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0))

# Integer ticks on x
ax.xaxis.set_major_locator(mtick.MaxNLocator(integer=True))


ax.legend()
plt.show()

## Evolution of Effiency

In [ ]:
def efficiency_level(sl: SimulationStepLog): 
    max_reward = sl.info["num_coins"] * np.max(np.array(list(sl.info["values"].values())))
    sum_rewards = np.sum(np.array(list(sl.rewards.values())))
    return sum_rewards / max_reward

def split_filter(sl: SimulationStepLog):
    info : dict = sl.info
    if "coins_given_to_self" in info and "num_coins" in info:
        return True
    return False

nb_iterations = len(rollout_trees)
fig, ax = plt.subplots()
efficiency_levels = [[gather_simulation_stats(path=path, filter=split_filter, stat_func=efficiency_level) for path in paths] for paths in main_rollouts_paths]
efficiency_levels = [np.mean(np.array(gls)) for gls in efficiency_levels]
ax.plot(np.arange(len(efficiency_levels)), efficiency_levels)

# --- Axes & formatting ---
ax.set_xlabel("Iteration")
ax.set_ylabel("Average Efficiency")

# Percent formatter — use xmax=100 if your stats are already in [0,100]
ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0))

# Integer ticks on x
ax.xaxis.set_major_locator(mtick.MaxNLocator(integer=True))

plt.show()

# Evolution of Reward


In [ ]:
def reward(agent_id: str, sl: SimulationStepLog): 
    return sl.rewards[agent_id]

nb_iterations = len(rollout_trees)
agent_ids = get_all_agents(rollout_trees[0][0])
print(agent_ids)
agent_greedy_levels = {}
fig, ax = plt.subplots()
for agent_id in agent_ids:
    stat_func = lambda sl: reward(agent_id, sl)
    avg_rewards = [[gather_simulation_stats(path=path, filter=lambda sl: True, stat_func=stat_func) for path in paths] for paths in main_rollouts_paths]
    avg_rewards = [np.mean(np.array(gls)) for gls in avg_rewards]
    ax.plot(np.arange(len(avg_rewards)), avg_rewards, label=agent_id)
    
# --- Axes & formatting ---
ax.set_xlabel("Iteration")
ax.set_ylabel("Average Reward")


# Integer ticks on x
ax.xaxis.set_major_locator(mtick.MaxNLocator(integer=True))


ax.legend()
plt.show()